<div>
<img src="./images/sunpy_logo.png" width="500" align="left"/>
</div>

# 1. Searching and downloading data with sunpy



In this notebook, an introduction to how you can search for and download data with sunpy. We will begin with an intoduction to `astropy.units` (which are used throughout the sunpy ecosystem), and then look about how to use `Fido` and build queries for data. In particular, this notebook will look at the following:

1. Introduction to `astropy.units`
2. Overview of `Fido` 
3. Constructing a data search query and inspecting it
4. More complex queries and the HEK
5. Extending Fido - the SOAR archive

In [2]:
import astropy.units as u

from sunpy.net import Fido, attrs as a
from sunpy.time import parse_time

# sunpy_soar and DKIST are affiliated packages of SunPy 
# and register the SOAR and DKIST clients
import sunpy_soar
#import dkist

import numpy as np

## 1.1 Astropy Units - a quick overview
[`astropy.units`](https://docs.astropy.org/en/stable/units/) provides a means to deal with and handle numbers/arrays etc that have an associated physical quantity (e.g. km, seconds, Kelvin). Throughout SunPy, any physical input or outputs is an [`astropy.Quantity`](https://docs.astropy.org/en/stable/units/quantity.html#quantity). Lets look at how we can create and convert between astropy units. Above we have imported `astropy.units` as `u`

In [3]:
distance_in_km = 10*u.km

In [4]:
distance_in_km

<Quantity 10. km>

In [5]:
distance_in_km.unit

Unit("km")

In [6]:
distance_in_km.value

10.0

We can convert between equivalent units

In [7]:
distance_in_km.cgs

<Quantity 1000000. cm>

In [15]:
distance_in_km.to(u.parsec)

<Quantity 3.24077929e-13 pc>

In [16]:
distance_in_km.to(u.Mm)

<Quantity 0.01 Mm>

However you can only convert between physical units that make sense for example:

In [10]:
#distance_in_km.to(u.second)

In [11]:
time_in_sec = 60*u.s

In [12]:
(distance_in_km/time_in_sec).unit

Unit("km / s")

In [19]:
(10*u.Angstrom).to(u.nm)

<Quantity 1. nm>

# 1.2 Overview of sunpy's Fido Unified Downloader
Fido is sunpy's interface for searching and downloading solar physics data.
It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.
You can also search and accesses multiple instruments and all available data providers in a single query.
It supplies a single, easy, consistent and extendable way to get most forms of solar physics data the community need.

For more information about Fido and how to use it check out the documentation on our website: https://docs.sunpy.org/en/stable/tutorial/acquiring_data/index.html

Fido offers access to data available through:

* VSO
* JSOC (through drms)
* Individual data providers from web accessible sources (http, ftp, etc)
* CDAWeb
* HEK
* HELIO
  
As described here Fido provides access to many sources of data through different clients, these clients can be defined inside sunpy or in other packages (e.g. DKIST data can be accessed using Fido through [DKIST User Tools](https://docs.dkist.nso.edu/projects/python-tools/en/latest/tutorial/2_search_and_asdf_download.html)).


#### Importantly, Solar Orbiter data can be accessed through the client defined in the `sunpy_soar` affiliated package.
The SOAR client is registered once we install `sunpy_soar` above. Without installing it, it wont be registered within Fido.

Lets first inspect the clients that are available through Fido:

In [20]:
Fido

Client,Description
CDAWEBClient,Provides access to query and download from the Coordinated Data Analysis Web (CDAWeb).
EVEClient,Provides access to Level 0CS Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to several GOES XRS files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
GONGClient,Provides access to the Magnetogram products of NSO-GONG synoptic Maps.
LYRAClient,Provides access to the LYRA/Proba2 data archive.
NOAAIndicesClient,Provides access to the NOAA solar cycle indices.
NOAAPredictClient,Provides access to the NOAA SWPC predicted sunspot Number and 10.7 cm radio flux values.
SRSClient,Provides access to the NOAA SWPC solar region summary data.


### Using attributes to search for data with Fido

Sunpy uses specified **attributes** to search for data using Fido. The range of these attributes is located in the `attrs` submodule. These `attr` parameters can be combined together to construct data search queries, such as searching over a certain time period, for data from a certain instrument with a certain wavelength etc.

Different clients and provides will have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`


Lets look at how these attributes work in more detail.

First we can look at `a.Time`, which is used to specify the timerange of a query.

In [21]:
a.Time("2022-03-28 11:00", "2022-03-28 14:00")

<sunpy.net.attrs.Time(2022-03-28 11:00:00.000, 2022-03-28 14:00:00.000)>

We can inspect the instrument attribute to see what instrument `attrs` are currently supported through sunpy. Here we can see the instrument name (i.e. the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.)

In [22]:
a.Instrument

Attribute Name,Client,Full Name,Description
aia,VSO,AIA,Atmospheric Imaging Assembly
bcs,VSO,BCS,Bragg Crystal Spectrometer
be_continuum,VSO,BE-Continuum,INAF-OACT Barra Equatoriale Continuum Instrument
be_halpha,VSO,BE-Halpha,INAF-OACT Barra Equatoriale Hα Instrument
bigbear,VSO,Big Bear,"Big Bear Solar Observatory, California TON and GONG+ sites"
caii,VSO,CAII,Kanzelhöhe Ca II k Instrument
cds,VSO,CDS,Coronal Diagnostic Spectrometer
celias,VSO,CELIAS,"Charge, Element, and Isotope Analysis System"
cerrotololo,VSO,Cerro Tololo,"Cerro Tololo, Chile GONG+ site"
chp,VSO,chp,Chromospheric Helium-I Imaging Photometer


sunpy also now provides tab completion to auto-fill the attribute name

In [23]:
a.Instrument.eit

<sunpy.net.attrs.Instrument(EIT: Extreme ultraviolet Imaging Telescope) object at 0x1057f5fd0>

To search for certain wavelengths, we need to specify the input as an `astropy.Quantity` which is a the combination of a value and an associated unit. This is something is universal in the sunpy stack - that every physical input/output is a `Quantity`.

In [24]:
a.Wavelength(17.1*u.angstrom)

<sunpy.net.attrs.Wavelength(17.1, 17.1, 'Angstrom')>

## 3. Constructing a search query
 ### A simple query

Lets create a simple query to search for data from AIA over a particular time period

In [17]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"))

In [25]:
# result

Now lets make our query a bit more specific, say, say we only want one wavelength band from AIA. This can be achieved by specifying the `Wavelength` attribute within the search. The `a.Wavelength` attribute is passed as an `astropy.Quantity`:

In [19]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(304*u.angstrom))

In [26]:
# result

We can further specify this query by choosing the cadence (time-sampling) of the data we want to search for and download. This can be achieved by using the Sample attribute. Similar to the Wavelength attributes, this needs to be an astropy Quantity. Lets further specify the search above to only search for data with a cadence of 2 minutes.

In [31]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(304*u.angstrom),
                     a.Sample(10*u.min))

In [29]:
# len(result[0])

## 1.3 Downloading the data

Now we can show how data that is queried above can be downloaded. Once the data you have searched for (and filtered etc) is constructed into a query using `Fido.search`, you can then easily download them using `Fido.fetch`.

The data is downloaded via asynchronous and parallel download streams (via parfive), and also allows for failed data downloads to be recognized so that files can be re-requested if not downloaded.

Lets now look at how a `UnifiedResponse` from a `Fido.search` can be passed to `Fido.fetch` to download the data

In [23]:
files = Fido.fetch(result)

Files Downloaded:   0%|          | 0/18 [00:00<?, ?file/s]

aia_lev1_304a_2022_03_28t11_40_05_15z_image_lev1.fits:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t11_10_05_13z_image_lev1.fits:   0%|          | 0.00/7.41M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t11_20_05_13z_image_lev1.fits:   0%|          | 0.00/7.42M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t11_00_05_13z_image_lev1.fits:   0%|          | 0.00/7.41M [00:00<?, ?B/s]

...

aia_lev1_304a_2022_03_28t13_10_05_13z_image_lev1.fits:   0%|          | 0.00/7.48M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t13_20_05_15z_image_lev1.fits:   0%|          | 0.00/7.48M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t13_30_05_13z_image_lev1.fits:   0%|          | 0.00/7.48M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t13_40_05_13z_image_lev1.fits:   0%|          | 0.00/7.47M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t13_50_05_13z_image_lev1.fits:   0%|          | 0.00/7.47M [00:00<?, ?B/s]

These files are downloaded to a local location set in the sunpy.config.file, which by default is ~/sunpy/data/. Fido.fetch returns a parfile.Results object which gives the path to where the files are downloaded to

In [24]:
print(files[0])

/Users/laura.hayes/sunpy/data/aia_lev1_304a_2022_03_28t11_00_05_13z_image_lev1.fits


You can also define what directory you want the files to be saved to by passing the directory path to the path keyword in Fido.fetch. For example, I want to download these files to a local directory `./AIA/<name_of_file>`

In [25]:
Fido.fetch(result, path="./{instrument}/{file}")

Files Downloaded:   0%|          | 0/18 [00:00<?, ?file/s]

aia_lev1_304a_2022_03_28t11_40_05_15z_image_lev1.fits:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t11_10_05_13z_image_lev1.fits:   0%|          | 0.00/7.41M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t11_20_05_13z_image_lev1.fits:   0%|          | 0.00/7.42M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t11_00_05_13z_image_lev1.fits:   0%|          | 0.00/7.41M [00:00<?, ?B/s]

...

aia_lev1_304a_2022_03_28t13_20_05_15z_image_lev1.fits:   0%|          | 0.00/7.48M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t13_30_05_13z_image_lev1.fits:   0%|          | 0.00/7.48M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t13_40_05_13z_image_lev1.fits:   0%|          | 0.00/7.47M [00:00<?, ?B/s]

aia_lev1_304a_2022_03_28t13_50_05_13z_image_lev1.fits:   0%|          | 0.00/7.47M [00:00<?, ?B/s]

['AIA/aia_lev1_304a_2022_03_28t11_00_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t11_10_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t11_20_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t11_30_05_15z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t11_40_05_15z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t11_50_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t12_00_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t12_10_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t12_20_05_12z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t12_30_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t12_40_05_14z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t12_50_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t13_00_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t13_10_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t13_20_05_15z_image_lev1.fits', 'AIA/aia_lev1_304a_2022_03_28t13_30_05_13z_image_lev1.fits', 'AIA/aia_lev1_304a_2022

## 1. 4 More complex queries

In addition to making a query to one client for one instrument, `Fido` allows the flexibility to search for data from multiple instruments, wavelengths, times etc, even when the data is being obtained through different clients.

This query can be constructed by using the pipe | operator, which joins queries together just like the OR operator.

Lets now make a query that searches for both GOES/XRS and AIA data over a particular time period

In [26]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument.xrs  | (a.Instrument.aia & a.Wavelength(304*u.angstrom)& a.Sample(10*u.min)))

In [27]:
len(result)

2

In [28]:
result

<sunpy.net.fido_factory.UnifiedResponse object at 0x122efb220>
Results from 2 Providers:

2 Results from the XRSClient:
Source: <13: https://umbra.nascom.nasa.gov/goes/fits 
13, 14, 15: https://satdat.ngdc.noaa.gov/sem/goes/data/science/ 
16, 17: https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/

       Start Time               End Time        Instrument SatelliteNumber  Physobs   Source Provider
----------------------- ----------------------- ---------- --------------- ---------- ------ --------
2022-03-28 00:00:00.000 2022-03-28 23:59:59.999        XRS              16 irradiance   GOES     NOAA
2022-03-28 00:00:00.000 2022-03-28 23:59:59.999        XRS              17 irradiance   GOES     NOAA

18 Results from the VSOClient:
Source: http://vso.stanford.edu/cgi-bin/search
Total estimated size: 1.22 Gbyte

       Start Time               End Time        Source Instrument ... Extent Type   Size                              Info                          
                                                                  ...              Mibyte                                                           
----------------------- ----------------------- ------ ---------- ... ----------- -------- ---------------------------------------------------------
2022-03-28 11:00:05.000 2022-03-28 11:00:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 11:10:05.000 2022-03-28 11:10:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 11:20:05.000 2022-03-28 11:20:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 11:30:05.000 2022-03-28 11:30:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 11:40:05.000 2022-03-28 11:40:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 11:50:05.000 2022-03-28 11:50:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 12:00:05.000 2022-03-28 12:00:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 12:10:05.000 2022-03-28 12:10:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 12:20:05.000 2022-03-28 12:20:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 12:30:05.000 2022-03-28 12:30:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 12:40:05.000 2022-03-28 12:40:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 12:50:05.000 2022-03-28 12:50:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 13:00:05.000 2022-03-28 13:00:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 13:10:05.000 2022-03-28 13:10:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 13:20:05.000 2022-03-28 13:20:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 13:30:05.000 2022-03-28 13:30:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 13:40:05.000 2022-03-28 13:40:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]
2022-03-28 13:50:05.000 2022-03-28 13:50:06.000    SDO        AIA ...    FULLDISK 64.64844 AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]

In [29]:
result[0]

Start Time,End Time,Instrument,SatelliteNumber,Physobs,Source,Provider
Time,Time,str3,int64,str10,str4,str4
2022-03-28 00:00:00.000,2022-03-28 23:59:59.999,XRS,16,irradiance,GOES,NOAA
2022-03-28 00:00:00.000,2022-03-28 23:59:59.999,XRS,17,irradiance,GOES,NOAA


In [30]:
result.all_colnames

['End Time',
 'Instrument',
 'Physobs',
 'Provider',
 'SatelliteNumber',
 'Source',
 'Start Time',
 'url']

Lets download the GOES XRS data first

In [31]:
Fido.fetch(result[0, 0], path="./")

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

sci_xrsf-l2-flx1s_g16_d20220328_v2-1-0.nc:   0%|          | 0.00/5.20M [00:00<?, ?B/s]

['sci_xrsf-l2-flx1s_g16_d20220328_v2-1-0.nc']

Now lets say we only want to download one AIA file at a particular time, we can also search the table for certain conditions. Lets say we just want the file that closest to 2022-03-28 12:00. 

In [32]:
(np.abs(result[1]["Start Time"] - parse_time("2022-03-28 12:00"))).argmin()

6

In [33]:
#Fido.fetch(result[1, 6], path="./")


## 1. 5 Using External Fido Clients 

Within `sunpy` core, we support a number of clients to common data providers. However, the `Fido` search interface is extensible such that external packages can write that their own clients that extend `Fido` in order to additional data sources. One such example is the `sunpy_soar` package which adds a client for the Solar Orbter Archive (SOAR).


## SOAR archive searching using sunpy!

In [34]:
import sunpy_soar


Note that after importing `sunpy_soar`, the SOAR is now listed as a client that `Fido` will search.

In [35]:
Fido

Client,Description
CDAWEBClient,Provides access to query and download from the Coordinated Data Analysis Web (CDAWeb).
EVEClient,Provides access to Level 0CS Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to the GOES XRS fits files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
GONGClient,Provides access to the Magnetogram products of NSO-GONG synoptic Maps.
LYRAClient,Provides access to the LYRA/Proba2 data archive.
NOAAIndicesClient,Provides access to the NOAA solar cycle indices.
NOAAPredictClient,Provides access to the NOAA SWPC predicted sunspot Number and 10.7 cm radio flux values.
SRSClient,Provides access to the NOAA SWPC solar region summary data.


In [42]:
eui_query = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                        a.soar.Product("EUI-FSI174-IMAGE"), 
                        a.Level(2))

In [43]:
#eui_query

In [44]:
Fido.fetch(eui_query, path="./{instrument}/{file}")

Files Downloaded:   0%|          | 0/18 [00:00<?, ?file/s]

solo_L2_eui-fsi174-image_20220328T132045222_V01.fits:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

solo_L2_eui-fsi174-image_20220328T134045224_V01.fits:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

solo_L2_eui-fsi174-image_20220328T135045225_V01.fits:   0%|          | 0.00/5.84M [00:00<?, ?B/s]

solo_L2_eui-fsi174-image_20220328T131045221_V01.fits:   0%|          | 0.00/5.87M [00:00<?, ?B/s]

...

solo_L2_eui-fsi174-image_20220328T113045210_V01.fits:   0%|          | 0.00/5.84M [00:00<?, ?B/s]

solo_L2_eui-fsi174-image_20220328T112045209_V01.fits:   0%|          | 0.00/5.86M [00:00<?, ?B/s]

solo_L2_eui-fsi174-image_20220328T111045208_V01.fits:   0%|          | 0.00/5.89M [00:00<?, ?B/s]

solo_L2_eui-fsi174-image_20220328T110045524_V01.fits:   0%|          | 0.00/5.89M [00:00<?, ?B/s]

['EUI/solo_L2_eui-fsi174-image_20220328T135045225_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T134045224_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T133045223_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T132045222_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T131045221_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T130045220_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T125045219_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T124045217_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T123045217_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T122045215_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T121045221_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T120045213_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T115045212_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T114045211_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T113045210_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T112045209_V01.fits', 'EUI/solo_L2_eui-fsi174-image_20220328T

We can also search for other data products, for example the Solar Orbiter MAG

In [39]:
mag_query = Fido.search(a.Time("2022-02-15", "2022-02-18"), 
                        a.soar.Product("MAG-RTN-NORMAL-1-MINUTE"), 
                        a.Level(2))

In [40]:
mag_query

Instrument,Data product,Level,Start time,End time,Data item ID,Filename,Filesize
,,,,,,,Mbyte
str3,str23,str2,str23,str23,str40,str48,float64
MAG,MAG-RTN-NORMAL-1-MINUTE,L2,2022-02-18 00:00:00.000,2022-02-19 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220218,solo_L2_mag-rtn-normal-1-minute_20220218_V01.cdf,0.02
MAG,MAG-RTN-NORMAL-1-MINUTE,L2,2022-02-17 00:00:00.000,2022-02-18 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220217,solo_L2_mag-rtn-normal-1-minute_20220217_V01.cdf,0.024
MAG,MAG-RTN-NORMAL-1-MINUTE,L2,2022-02-16 00:00:00.000,2022-02-17 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220216,solo_L2_mag-rtn-normal-1-minute_20220216_V01.cdf,0.032
MAG,MAG-RTN-NORMAL-1-MINUTE,L2,2022-02-15 00:00:00.000,2022-02-16 00:00:00.000,solo_L2_mag-rtn-normal-1-minute_20220215,solo_L2_mag-rtn-normal-1-minute_20220215_V01.cdf,0.033


In [41]:
ff = Fido.fetch(mag_query, path="./")

Files Downloaded:   0%|          | 0/4 [00:00<?, ?file/s]

solo_L2_mag-rtn-normal-1-minute_20220216_V01.cdf:   0%|          | 0.00/32.2k [00:00<?, ?B/s]

solo_L2_mag-rtn-normal-1-minute_20220218_V01.cdf:   0%|          | 0.00/20.0k [00:00<?, ?B/s]

solo_L2_mag-rtn-normal-1-minute_20220217_V01.cdf:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

solo_L2_mag-rtn-normal-1-minute_20220215_V01.cdf:   0%|          | 0.00/32.8k [00:00<?, ?B/s]

# Accessing data from the CDAWeb with sunpy - which is very helpful for in-situ data

There is also a CDAWeb client within sunpy. CDAWeb data can be accessed when the `cdaweb.Dataset` attribute is provided to the search.

The data available from the SOAR is also available from the CDAWeb. You may be used to working with this (especially if you mainly work with in-situ observations), so lets go through how the data can also be accessed this way. This is handy, as you can also access many other in-situ measurements from this too.

In [ ]:
a.cdaweb.Dataset

In [ ]:
res_cdaw = Fido.search(a.Time("2022-03-25", "2022-03-26"), 
                       a.cdaweb.Dataset('SOLO_L2_MAG-RTN-NORMAL-1-MINUTE'))

In [ ]:
res_cdaw